# Vis-Sieve Troubleshooting

The purpose of this notebook is to troubleshoot issues we see in Vis-Sieve.

## 01_get-urls

### ISSUE 1 [RESOLVED] - Obtaining a subset of results from our OpenAlex queries, and different subset numbers depending on output being JSON or DB.

[RESOLVED - Devin had a setting turned to 'True' for testing--to stop at certain number of pages--we set to 'False']


**Comparing Outputs for the year 2019**

OpenAlex BROWSER OUTPUT: 6959  
* Method: https://api.openalex.org/works?filter=publication_year:2019,institutions.ror:00hx57361&cursor=*
* first (work)id: "https://openalex.org/W3104062568"


JSON OUTPUT: works - 6959; authors - 21420 
* Method:  
python -m 01_get-urls.get-urls \
2019 2019 \
--ror 00hx57361 \
--email c.roe-raymond@princeton.edu \
--output urls_2019.json \
-a \
--content_root results_json/ 
* tqdm bar said #/35

DB OUTPUT (no database files existed):  works - 400; authors - 940
* Method:  
python -m 01_get-urls.get-urls \
2019 2019 \
--ror 00hx57361 \
--email c.roe-raymond@princeton.edu \
--output urls_2019.json \
-a \
--database database/publications_princeton_2019.db \
--content_root results_json/ 
* tqdm bar said #/400
* first (work)id: 2223092947


### ISSUE 2 - Check validity/duplcation within results from 01_get-urls. 

In the first 20 lines, I already see a paper's title possibly 6 times. See ids:

+ 4399008018
+ 4399010284
+ 4399012787
+ 4399015917
+ 4399016637
+ 4399018608

### ISSUE 3 - [NOT URGENT] What happens when you run 01_get-urls multiple times; do you get duplicate entries? (testing year 2023)

[NOT URGENT - For now let's just delete the files and start over each time.]

When running file to database, getting error:
```
Duplicate author in Recent results from Borexino on solar neutrinos: Ö. Penek
 88%|████████████████████████████████████████████████████████████████████████████████████████████████████████████▌              | 30/34 [00:46<00:05,  1.37s/it]Duplicate author in Dark Energy Survey Deep Field photometric redshift performance and training incompleteness assessment: J. De Vicente
Duplicate author in Author Correction: Large composite fermion effective mass at filling factor 5/2: L. N. Pfeiffer
Duplicate author in High-order coronagraphic wavefront control with algorithmic differentiation: first experimental demonstration: Marshall D. Perrin
 91%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏          | 31/34 [00:47<00:03,  1.29s/it]Duplicate author in Index: Christy Wampole
Duplicate author in Index: Christy Wampole
Duplicate author in Recommendations for Further Reading: Christy Wampole
Duplicate author in Recommendations for Further Reading: Christy Wampole
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:49<00:00,  1.46s/it]
processing publications
 66%|██████████████████████████████████████████████████████████████████████████████                                         | 4341/6622 [02:13<01:10, 32.58it/s]
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/csimao/Library/CloudStorage/OneDrive-PrincetonUniversity/projects/personal_projs/vis_sieve_princeton/vis-sieve-org/current/01_get-urls/get-urls.py", line 313, in <module>
    populate_database(args.database, args.ror, args.email, range(int(args.first_year), int(args.last_year)+1), args.content_root, args.output, args.silent)
  File "/Users/csimao/Library/CloudStorage/OneDrive-PrincetonUniversity/projects/personal_projs/vis_sieve_princeton/vis-sieve-org/current/01_get-urls/get-urls.py", line 218, in populate_database
    pub_title = pub["title"][:200].replace("'", "")
                ~~~~~~~~~~~~^^^^^^
TypeError: 'NoneType' object is not subscriptable
```

After running 

```
# must run from Carolina's 'current' folder
cd current
python -m 01_get-urls.get-urls \
2023 2023 \
--ror 00hx57361 \
--email c.roe-raymond@princeton.edu \
--output urls_2023.json \
-a \
--database database/publications_princeton_2023.db \
--content_root results_json/ 
```
ONE TIME
*********************************
AUTHOR TABLE
on.sql(f"""
...     SELECT *
...     FROM {table_name}
...     """)
┌────────────┬───────────────────────┐
│     id     │         name          │
│   int64    │        varchar        │
├────────────┼───────────────────────┤
│ 5042307561 │ H. Vincent Poor       │
│ 5009751100 │ Gerard Joey Broussard │
│ 5032752482 │ Samuel S.‐H. Wang     │
│ 5083898813 │ Connelly Barnes       │
│ 5034842939 │ Adam Finkelstein      │
│ 5014376782 │ Matteo Cantiello      │
│ 5063345981 │ Sayash Kapoor         │
│ 5058102069 │ Arvind Narayanan      │
│ 5022811687 │ Olga Russakovsky      │
│ 5014363474 │ Caroline Bartman      │
│      ·     │        ·              │
│      ·     │        ·              │
│      ·     │        ·              │
│ 5092218660 │ Sebastián Rojas Cabal │
│ 5046646074 │ Hayrettin Sağlam      │
│ 5043372796 │ Lauren L. Emberson    │
│ 5025640421 │ Alexander Göbel       │
│ 5090200106 │ Emily A. Carter       │
│ 5090200106 │ Emily A. Carter       │
│ 5005406471 │ Gillian Frank         │
│ 5008697973 │ Harold James          │
│ 5102725201 │ Katja Guenther        │
│ 5071860424 │ Yana Prymachenko      │
├────────────┴───────────────────────┤
│ 8927 rows (20 shown)     2 columns │
└────────────────────────────────────┘

PAPER TABLE

┌────────────┬──────────────────────┬──────────────────────┬──────────────────┬───────────────────────────────────────────────────────────────────────────┬───────────┬───────────────┬──────────┬──────────┐
│     id     │        title         │         doi          │ publication_date │                                  oa_url                                   │ oa_status │ pdf_retrieved │ pdf_path │ inst_id  │
│   int64    │       varchar        │       varchar        │       date       │                                  varchar                                  │  varchar  │    varchar    │ varchar  │  int64   │
├────────────┼──────────────────────┼──────────────────────┼──────────────────┼───────────────────────────────────────────────────────────────────────────┼───────────┼───────────────┼──────────┼──────────┤
│ 4322576964 │ On the Road to 6G:…  │ https://doi.org/10…  │ 2023-01-01       │ https://hal.science/hal-04261267/document                                 │ green     │ NULL          │ NULL     │ 20089843 │
│ 4324344199 │ Fast and sensitive…  │ https://doi.org/10…  │ 2023-03-15       │ https://www.nature.com/articles/s41586-023-05828-9.pdf                    │ hybrid    │ NULL          │ NULL     │ 20089843 │
│ 4385490375 │ PatchMatch: A Rand…  │ https://doi.org/10…  │ 2023-08-01       │ https://doi.org/10.1145/3596711.3596777                                   │ bronze    │ NULL          │ NULL     │ 20089843 │
│ 4321790088 │ Modules for Experi…  │ https://doi.org/10…  │ 2023-02-23       │ https://iopscience.iop.org/article/10.3847/1538-4365/acae8d/pdf           │ gold      │ NULL          │ NULL     │ 20089843 │
│ 4385576721 │ Leakage and the re…  │ https://doi.org/10…  │ 2023-08-04       │ http://www.cell.com/article/S2666389923001599/pdf                         │ gold      │ NULL          │ NULL     │ 20089843 │
│ 4380769213 │ Art and the scienc…  │ https://doi.org/10…  │ 2023-06-15       │ https://doi.org/10.1126/science.adh4451                                   │ green     │ NULL          │ NULL     │ 20089843 │
│ 4318771331 │ Slow TCA flux and …  │ https://doi.org/10…  │ 2023-02-01       │ https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10288502                     │ green     │ NULL          │ NULL     │ 20089843 │
│ 4315436844 │ Early Release Scie…  │ https://doi.org/10…  │ 2023-01-09       │ https://www.nature.com/articles/s41586-022-05677-y.pdf                    │ hybrid    │ NULL          │ NULL     │ 20089843 │
│ 4324344255 │ Spatial epigenome–…  │ https://doi.org/10…  │ 2023-03-15       │ https://www.nature.com/articles/s41586-023-05795-1.pdf                    │ hybrid    │ NULL          │ NULL     │ 20089843 │
│ 4382536268 │ Machine Learning i…  │ https://doi.org/10…  │ 2023-06-29       │ None                                                                      │ closed    │ NULL          │ NULL     │ 20089843 │
│      ·     │          ·           │          ·           │     ·            │  ·                                                                        │   ·       │  ·            │  ·       │     ·    │
│      ·     │          ·           │          ·           │     ·            │  ·                                                                        │   ·       │  ·            │  ·       │     ·    │
│      ·     │          ·           │          ·           │     ·            │  ·                                                                        │   ·       │  ·            │  ·       │     ·    │
│ 4381429392 │ Making Sense of Vi…  │ https://doi.org/10…  │ 2023-01-01       │ https://online.ucpress.edu/gp/article-pdf/4/1/74935/779187/globalperspe…  │ hybrid    │ NULL          │ NULL     │ 20089843 │
│ 4381431380 │ Challenges in elec…  │ https://doi.org/10…  │ 2023-05-30       │ None                                                                      │ closed    │ NULL          │ NULL     │ 20089843 │
│ 4381516797 │ Scalp Surface-Base…  │ https://doi.org/10…  │ 2023-01-01       │ None                                                                      │ closed    │ NULL          │ NULL     │ 20089843 │
│ 4381550427 │ How the position o…  │ https://doi.org/10…  │ 2023-06-21       │ https://doi.org/10.16995/glossa.9273                                      │ gold      │ NULL          │ NULL     │ 20089843 │
│ 4381597715 │ A Tribute to Micha…  │ https://doi.org/10…  │ 2023-06-22       │ https://doi.org/10.1021/acs.jpcb.3c03044                                  │ bronze    │ NULL          │ NULL     │ 20089843 │
│ 4381598485 │ A Tribute to Micha…  │ https://doi.org/10…  │ 2023-06-22       │ https://doi.org/10.1021/acs.jpca.3c03071                                  │ bronze    │ NULL          │ NULL     │ 20089843 │
│ 4381610806 │ Sex with the Sound…  │ https://doi.org/10…  │ 2023-06-01       │ None                                                                      │ closed    │ NULL          │ NULL     │ 20089843 │
│ 4381610845 │ Stefan J. Link. <i…  │ https://doi.org/10…  │ 2023-06-01       │ None                                                                      │ closed    │ NULL          │ NULL     │ 20089843 │
│ 4381610931 │ Camille Robcis. <i…  │ https://doi.org/10…  │ 2023-06-01       │ None                                                                      │ closed    │ NULL          │ NULL     │ 20089843 │
│ 4381664217 │ The transformation…  │ https://doi.org/10…  │ 2023-06-01       │ https://sciendo.com/pdf/10.2478/njms-2023-0005                            │ diamond   │ NULL          │ NULL     │ 20089843 │
├────────────┴──────────────────────┴──────────────────────┴──────────────────┴───────────────────────────────────────────────────────────────────────────┴───────────┴───────────────┴──────────┴──────────┤
│ 4339 rows (20 shown)                                                                                                                                                                            9 columns


SECOND TIME
*********************************

AUTHOR TABLE
┌────────────┬───────────────────────┐
│     id     │         name          │
│   int64    │        varchar        │
├────────────┼───────────────────────┤
│ 5042307561 │ H. Vincent Poor       │
│ 5009751100 │ Gerard Joey Broussard │
│ 5032752482 │ Samuel S.‐H. Wang     │
│ 5083898813 │ Connelly Barnes       │
│ 5034842939 │ Adam Finkelstein      │
│ 5014376782 │ Matteo Cantiello      │
│ 5063345981 │ Sayash Kapoor         │
│ 5058102069 │ Arvind Narayanan      │
│ 5022811687 │ Olga Russakovsky      │
│ 5014363474 │ Caroline Bartman      │
│      ·     │       ·               │
│      ·     │       ·               │
│      ·     │       ·               │
│ 5071366167 │ Sagi Jaffe‐Dax        │
│ 5103150223 │ Christine Potter      │
│ 5063756370 │ Tiffany S. Leung      │
│ 5043372796 │ Lauren L. Emberson    │
│ 5077673530 │ Casey Lew‐Williams    │
│ 5017091710 │ J. H. Clark           │
│ 5084277289 │ Mohamed Seif          │
│ 5042307561 │ H. Vincent Poor       │
│ 5084277289 │ Mohamed Seif          │
│ 5042307561 │ H. Vincent Poor       │
├────────────┴───────────────────────┤
│ 9412 rows (20 shown)     2 columns │
└────────────────────────────────────┘

PAPER TABLE
┌────────────┬──────────────────────────────────────┬───────────────────────────────────────────────┬──────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────┬───────────────┬──────────┬──────────┐
│     id     │                title                 │                      doi                      │ publication_date │                                               oa_url                                                │ oa_status │ pdf_retrieved │ pdf_path │ inst_id  │
│   int64    │               varchar                │                    varchar                    │       date       │                                               varchar                                               │  varchar  │    varchar    │ varchar  │  int64   │
├────────────┼──────────────────────────────────────┼───────────────────────────────────────────────┼──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────┼───────────────┼──────────┼──────────┤
│ 4322576964 │ On the Road to 6G: Visions, Requir…  │ https://doi.org/10.1109/comst.2023.3249835    │ 2023-01-01       │ https://hal.science/hal-04261267/document                                                           │ green     │ NULL          │ NULL     │ 20089843 │
│ 4324344199 │ Fast and sensitive GCaMP calcium i…  │ https://doi.org/10.1038/s41586-023-05828-9    │ 2023-03-15       │ https://www.nature.com/articles/s41586-023-05828-9.pdf                                              │ hybrid    │ NULL          │ NULL     │ 20089843 │
│ 4385490375 │ PatchMatch: A Randomized Correspon…  │ https://doi.org/10.1145/3596711.3596777       │ 2023-08-01       │ https://doi.org/10.1145/3596711.3596777                                                             │ bronze    │ NULL          │ NULL     │ 20089843 │
│ 4321790088 │ Modules for Experiments in Stellar…  │ https://doi.org/10.3847/1538-4365/acae8d      │ 2023-02-23       │ https://iopscience.iop.org/article/10.3847/1538-4365/acae8d/pdf                                     │ gold      │ NULL          │ NULL     │ 20089843 │
│ 4385576721 │ Leakage and the reproducibility cr…  │ https://doi.org/10.1016/j.patter.2023.100804  │ 2023-08-04       │ http://www.cell.com/article/S2666389923001599/pdf                                                   │ gold      │ NULL          │ NULL     │ 20089843 │
│ 4380769213 │ Art and the science of generative AI │ https://doi.org/10.1126/science.adh4451       │ 2023-06-15       │ https://doi.org/10.1126/science.adh4451                                                             │ green     │ NULL          │ NULL     │ 20089843 │
│ 4318771331 │ Slow TCA flux and ATP production i…  │ https://doi.org/10.1038/s41586-022-05661-6    │ 2023-02-01       │ https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10288502                                               │ green     │ NULL          │ NULL     │ 20089843 │
│ 4315436844 │ Early Release Science of the exopl…  │ https://doi.org/10.1038/s41586-022-05677-y    │ 2023-01-09       │ https://www.nature.com/articles/s41586-022-05677-y.pdf                                              │ hybrid    │ NULL          │ NULL     │ 20089843 │
│ 4324344255 │ Spatial epigenome–transcriptome co…  │ https://doi.org/10.1038/s41586-023-05795-1    │ 2023-03-15       │ https://www.nature.com/articles/s41586-023-05795-1.pdf                                              │ hybrid    │ NULL          │ NULL     │ 20089843 │
│ 4382536268 │ Machine Learning in Environmental …  │ https://doi.org/10.1021/acs.est.3c00026       │ 2023-06-29       │ None                                                                                                │ closed    │ NULL          │ NULL     │ 20089843 │
│      ·     │                  ·                   │                   ·                           │     ·            │  ·                                                                                                  │   ·       │  ·            │  ·       │     ·    │
│      ·     │                  ·                   │                   ·                           │     ·            │  ·                                                                                                  │   ·       │  ·            │  ·       │     ·    │
│      ·     │                  ·                   │                   ·                           │     ·            │  ·                                                                                                  │   ·       │  ·            │  ·       │     ·    │
│ 4388704021 │ Self-regulation and alcohol consum…  │ https://doi.org/10.31234/osf.io/96zac         │ 2023-11-14       │ https://osf.io/96zac/download                                                                       │ green     │ NULL          │ NULL     │ 20089843 │
│ 4388709522 │ Damping of MHD turbulence in a par…  │ https://doi.org/10.1093/mnras/stad3493        │ 2023-11-09       │ https://academic.oup.com/mnras/advance-article-pdf/doi/10.1093/mnras/stad3493/53406058/stad3493.pdf │ hybrid    │ NULL          │ NULL     │ 20089843 │
│ 4388802438 │ The Political economy of the minim…  │ https://doi.org/10.1016/j.labeco.2023.102463  │ 2023-11-19       │ None                                                                                                │ closed    │ NULL          │ NULL     │ 20089843 │
│ 4388895240 │ Who Bears the Costs of Inflation? …  │ https://doi.org/10.2139/ssrn.4640891          │ 2023-01-01       │ https://www.econstor.eu/bitstream/10419/297317/1/ecb.wp2877.pdf                                     │ green     │ NULL          │ NULL     │ 20089843 │
│ 4388895409 │ Habitat selection by Grevys zebra …  │ https://doi.org/10.1111/aje.13229             │ 2023-11-20       │ https://onlinelibrary.wiley.com/doi/pdfdirect/10.1111/aje.13229                                     │ hybrid    │ NULL          │ NULL     │ 20089843 │
│ 4388901017 │ Periodical Cicadas and the Abundan…  │ https://doi.org/10.55283/jhk.12434            │ 2023-11-22       │ https://journalhistoryknowledge.org/article/download/12434/19956                                    │ diamond   │ NULL          │ NULL     │ 20089843 │
│ 4388901056 │ The Influence of Memory on Visual …  │ https://doi.org/10.1111/cogs.13381            │ 2023-11-01       │ https://onlinelibrary.wiley.com/doi/pdfdirect/10.1111/cogs.13381                                    │ hybrid    │ NULL          │ NULL     │ 20089843 │
│ 4388906895 │ Zonally asymmetric component of su…  │ https://doi.org/10.1038/s41612-023-00522-z    │ 2023-11-22       │ https://www.nature.com/articles/s41612-023-00522-z.pdf                                              │ gold      │ NULL          │ NULL     │ 20089843 │
│ 4388913133 │ Over-the-air Aggregation-based Fed…  │ https://doi.org/10.36227/techrxiv.24574768    │ 2023-11-22       │ https://www.techrxiv.org/doi/pdf/10.36227/techrxiv.24574768.v1                                      │ green     │ NULL          │ NULL     │ 20089843 │
│ 4388913624 │ Over-the-air Aggregation-based Fed…  │ https://doi.org/10.36227/techrxiv.24574768.v1 │ 2023-11-22       │ https://www.techrxiv.org/doi/pdf/10.36227/techrxiv.24574768.v1                                      │ green     │ NULL          │ NULL     │ 20089843 │
├────────────┴──────────────────────────────────────┴───────────────────────────────────────────────┴──────────────────┴─────────────────────────────────────────────────────────────────────────────────────────────────────┴───────────┴───────────────┴──────────┴──────────┤
│ 4549 rows (20 shown)                                                                                                                                                                                                                                               9 columns │
└──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

THIRD TIME
*********************************

AUTHORS TABLE
┌────────────┬───────────────────────┐
│     id     │         name          │
│   int64    │        varchar        │
├────────────┼───────────────────────┤
│ 5042307561 │ H. Vincent Poor       │
│ 5009751100 │ Gerard Joey Broussard │
│ 5032752482 │ Samuel S.‐H. Wang     │
│ 5083898813 │ Connelly Barnes       │
│ 5034842939 │ Adam Finkelstein      │
│ 5014376782 │ Matteo Cantiello      │
│ 5063345981 │ Sayash Kapoor         │
│ 5058102069 │ Arvind Narayanan      │
│ 5022811687 │ Olga Russakovsky      │
│ 5014363474 │ Caroline Bartman      │
│      ·     │       ·               │
│      ·     │       ·               │
│      ·     │       ·               │
│ 5071366167 │ Sagi Jaffe‐Dax        │
│ 5103150223 │ Christine Potter      │
│ 5063756370 │ Tiffany S. Leung      │
│ 5043372796 │ Lauren L. Emberson    │
│ 5077673530 │ Casey Lew‐Williams    │
│ 5017091710 │ J. H. Clark           │
│ 5084277289 │ Mohamed Seif          │
│ 5042307561 │ H. Vincent Poor       │
│ 5084277289 │ Mohamed Seif          │
│ 5042307561 │ H. Vincent Poor       │
├────────────┴───────────────────────┤
│ 9412 rows (20 shown)     2 columns │
└────────────────────────────────────┘

PAPER TABLE

┌────────────┬──────────────────────────────────────┬───────────────────────────────────────────────┬──────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────┬───────────────┬──────────┬──────────┐
│     id     │                title                 │                      doi                      │ publication_date │                                               oa_url                                                │ oa_status │ pdf_retrieved │ pdf_path │ inst_id  │
│   int64    │               varchar                │                    varchar                    │       date       │                                               varchar                                               │  varchar  │    varchar    │ varchar  │  int64   │
├────────────┼──────────────────────────────────────┼───────────────────────────────────────────────┼──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────┼───────────────┼──────────┼──────────┤
│ 4322576964 │ On the Road to 6G: Visions, Requir…  │ https://doi.org/10.1109/comst.2023.3249835    │ 2023-01-01       │ https://hal.science/hal-04261267/document                                                           │ green     │ NULL          │ NULL     │ 20089843 │
│ 4324344199 │ Fast and sensitive GCaMP calcium i…  │ https://doi.org/10.1038/s41586-023-05828-9    │ 2023-03-15       │ https://www.nature.com/articles/s41586-023-05828-9.pdf                                              │ hybrid    │ NULL          │ NULL     │ 20089843 │
│ 4385490375 │ PatchMatch: A Randomized Correspon…  │ https://doi.org/10.1145/3596711.3596777       │ 2023-08-01       │ https://doi.org/10.1145/3596711.3596777                                                             │ bronze    │ NULL          │ NULL     │ 20089843 │
│ 4321790088 │ Modules for Experiments in Stellar…  │ https://doi.org/10.3847/1538-4365/acae8d      │ 2023-02-23       │ https://iopscience.iop.org/article/10.3847/1538-4365/acae8d/pdf                                     │ gold      │ NULL          │ NULL     │ 20089843 │
│ 4385576721 │ Leakage and the reproducibility cr…  │ https://doi.org/10.1016/j.patter.2023.100804  │ 2023-08-04       │ http://www.cell.com/article/S2666389923001599/pdf                                                   │ gold      │ NULL          │ NULL     │ 20089843 │
│ 4380769213 │ Art and the science of generative AI │ https://doi.org/10.1126/science.adh4451       │ 2023-06-15       │ https://doi.org/10.1126/science.adh4451                                                             │ green     │ NULL          │ NULL     │ 20089843 │
│ 4318771331 │ Slow TCA flux and ATP production i…  │ https://doi.org/10.1038/s41586-022-05661-6    │ 2023-02-01       │ https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10288502                                               │ green     │ NULL          │ NULL     │ 20089843 │
│ 4315436844 │ Early Release Science of the exopl…  │ https://doi.org/10.1038/s41586-022-05677-y    │ 2023-01-09       │ https://www.nature.com/articles/s41586-022-05677-y.pdf                                              │ hybrid    │ NULL          │ NULL     │ 20089843 │
│ 4324344255 │ Spatial epigenome–transcriptome co…  │ https://doi.org/10.1038/s41586-023-05795-1    │ 2023-03-15       │ https://www.nature.com/articles/s41586-023-05795-1.pdf                                              │ hybrid    │ NULL          │ NULL     │ 20089843 │
│ 4382536268 │ Machine Learning in Environmental …  │ https://doi.org/10.1021/acs.est.3c00026       │ 2023-06-29       │ None                                                                                                │ closed    │ NULL          │ NULL     │ 20089843 │
│      ·     │                  ·                   │                   ·                           │     ·            │  ·                                                                                                  │   ·       │  ·            │  ·       │     ·    │
│      ·     │                  ·                   │                   ·                           │     ·            │  ·                                                                                                  │   ·       │  ·            │  ·       │     ·    │
│      ·     │                  ·                   │                   ·                           │     ·            │  ·                                                                                                  │   ·       │  ·            │  ·       │     ·    │
│ 4388704021 │ Self-regulation and alcohol consum…  │ https://doi.org/10.31234/osf.io/96zac         │ 2023-11-14       │ https://osf.io/96zac/download                                                                       │ green     │ NULL          │ NULL     │ 20089843 │
│ 4388709522 │ Damping of MHD turbulence in a par…  │ https://doi.org/10.1093/mnras/stad3493        │ 2023-11-09       │ https://academic.oup.com/mnras/advance-article-pdf/doi/10.1093/mnras/stad3493/53406058/stad3493.pdf │ hybrid    │ NULL          │ NULL     │ 20089843 │
│ 4388802438 │ The Political economy of the minim…  │ https://doi.org/10.1016/j.labeco.2023.102463  │ 2023-11-19       │ None                                                                                                │ closed    │ NULL          │ NULL     │ 20089843 │
│ 4388895240 │ Who Bears the Costs of Inflation? …  │ https://doi.org/10.2139/ssrn.4640891          │ 2023-01-01       │ https://www.econstor.eu/bitstream/10419/297317/1/ecb.wp2877.pdf                                     │ green     │ NULL          │ NULL     │ 20089843 │
│ 4388895409 │ Habitat selection by Grevys zebra …  │ https://doi.org/10.1111/aje.13229             │ 2023-11-20       │ https://onlinelibrary.wiley.com/doi/pdfdirect/10.1111/aje.13229                                     │ hybrid    │ NULL          │ NULL     │ 20089843 │
│ 4388901017 │ Periodical Cicadas and the Abundan…  │ https://doi.org/10.55283/jhk.12434            │ 2023-11-22       │ https://journalhistoryknowledge.org/article/download/12434/19956                                    │ diamond   │ NULL          │ NULL     │ 20089843 │
│ 4388901056 │ The Influence of Memory on Visual …  │ https://doi.org/10.1111/cogs.13381            │ 2023-11-01       │ https://onlinelibrary.wiley.com/doi/pdfdirect/10.1111/cogs.13381                                    │ hybrid    │ NULL          │ NULL     │ 20089843 │
│ 4388906895 │ Zonally asymmetric component of su…  │ https://doi.org/10.1038/s41612-023-00522-z    │ 2023-11-22       │ https://www.nature.com/articles/s41612-023-00522-z.pdf                                              │ gold      │ NULL          │ NULL     │ 20089843 │(year)
│ 4388913133 │ Over-the-air Aggregation-based Fed…  │ https://doi.org/10.36227/techrxiv.24574768    │ 2023-11-22       │ https://www.techrxiv.org/doi/pdf/10.36227/techrxiv.24574768.v1                                      │ green     │ NULL          │ NULL     │ 20089843 │
│ 4388913624 │ Over-the-air Aggregation-based Fed…  │ https://doi.org/10.36227/techrxiv.24574768.v1 │ 2023-11-22       │ https://www.techrxiv.org/doi/pdf/10.36227/techrxiv.24574768.v1                                      │ green     │ NULL          │ NULL     │ 20089843 │
├────────────┴──────────────────────────────────────┴───────────────────────────────────────────────┴──────────────────┴─────────────────────────────────────────────────────────────────────────────────────────────────────┴───────────┴───────────────┴──────────┴──────────┤
│ 4549 rows (20 shown)                                                                                                                                                                                                                                               9 columns │
└──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

Issue - I do not see the final ids in the API call for this year in a browser...  
Also, we need some text to make sure duplicates aren't added to these lists when we run the commands over and over  
Maybe try to have "none" type skip out of loop and list into a csv?  

Now going to run with .db, just to get json file...

**Comparing Outputs for the year 2023**

OpenAlex BROWSER OUTPUT: 6625  
* Method: https://api.openalex.org/works?filter=publication_year:2023,institutions.ror:00hx57361&cursor=*


JSON OUTPUT: works - 6625; authors - 28106
* Method:  
```
python -m 01_get-urls.get-urls \
2023 2023 \
--ror 00hx57361 \
--email c.roe-raymond@princeton.edu \
--output urls_2023.json \
-a \
--content_root results_json/ 
```

Create a csv file with institution author names and work ids.

In [2]:
# Create csv file with institution author names and work id
import json
import pandas as pd

year = "2023"

# define data
data = json.load(open(f"results_json/urls_{year}.json"))

# create csv for output - choose one or the other, comment/uncomment break accordingly in loop
#file = open("../data/authors_2022slim.csv", "w")
file = open(f"results_json/urls_{year}.csv", "w") 

# add header to .csv file
file.write("index_workObjInWorksJSON,author,workID,author_workID\n")

for work in data:
        # grab index of work object
        workID_index = data.index(work)
        for author in work["authorships"]:
            institutions = author["institutions"]
            for inst in institutions:
                if "princeton university" in inst["display_name"].lower():
                    # grab author's name from our institution for this work
                    name = author["author"]["display_name"]
                    # grab workid
                    id = work["id"]
                    name_id = name + "_" + id
                    # create line to write to csv
                    line = str(workID_index) + ", " + name + ", " + id + ", " + name_id + "\n"
                    file.write(line)
                #break
                
file.close()      